In [54]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Modelado y evaluación
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Configuración warnings
import warnings
warnings.filterwarnings('once')

In [55]:
df_total = pd.read_pickle('../../../datos_finales/archivo_total_est_enc2.pkl')
df_total.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,total_usuarios
0,1.0,1.0,1.0,1.04,1.00,2.23,-0.486274,0.855582,-0.208909,985
1,1.0,1.0,1.0,1.01,1.05,2.23,-0.425690,0.332722,0.687550,801


In [56]:
X = df_total.drop("total_usuarios", axis = 1)
y = df_total["total_usuarios"]

In [57]:
xg_reg = XGBRegressor()

In [58]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [59]:
xg_reg.fit(x_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [60]:
y_pred_test_gb = xg_reg.predict(x_test)
y_pred_train_gb = xg_reg.predict(x_train)


In [61]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    """
    Esta función nos dice las métricas de nuestro modelo.
    Args:
        y_test (Serie): el test de nuestra variable respuesta.
        y_train (Serie): el train de nuestra variable respuesta.
        y_test_pred (Serie): la predicción para el test de nuestra variable respuesta.
        y_train_pred (Serie): la predicción para el train de nuestra variable respuesta.
        tipo_modelo (str): nombre que le damos al modelo.
    Returns:
        dataframe con todas las métricas de nuestro modelo.
    """
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [62]:
dt_results = metricas(y_test, y_train, y_pred_test_gb, y_pred_train_gb, "Xgboost")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,435.921474,345647.104291,587.917600,0.899018,test,Xgboost
1,16.203558,552.335407,23.501817,0.999855,train,Xgboost


Este modelo nos devuelve buenas métricas para R2 pero se observa claramente el overfitting. 